# Predicting GroundState Energy from Coulomb Matrices

This dataset contains ground state energies of 16,242 molecules calculated by quantum mechanical simulations.

The data contains 1277 columns. The first 1275 columns are entries in the Coulomb matrix that act as molecular features. The 1276th column is the Pubchem Id where the molecular structures are obtained. The 1277th column is the atomization energy calculated by simulations using the Quantum Espresso package.

In the csv file, the first column (X1) is the data index.

The dataset was used for a [publication using a tree based ML Framework](https://arxiv.org/pdf/1609.07124.pdf).

## Getting the data

We create a class that inherits from torch's `Dataset` class to create our own dataset object. It must include the three given methods.

In [3]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

class GS_from_CM_Dataset(Dataset):
    
    def __init__(self, path_to_data, transform=None):
        
        self.path_to_data = path_to_data
        if os.path.exists(path_to_data):
            self.data = pd.read_csv(path_to_data)
        else:
            raise FileExistsError('No dataset found')
        self.transform = transform

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        X = self.data.drop(columns=['id', 'pubchem_id', 'Eat'])
        y = self.data['Eat']
        
        energy = self.data.iloc[idx, -1]
        CMat = self.data.iloc[idx, 1:-2]

        # sample = {'CMat': CMat, 'energy': energy}
        
        # if self.transform:
        #     sample = self.transform(sample)
            
        # return sample
        return torch.tensor(CMat).to(torch.float), torch.tensor(energy).to(torch.float)

In [4]:
dataset = GS_from_CM_Dataset(path_to_data='data/GS_from_CM/GS_from_CM_data.csv')

# Just checking that everything works fine
print(len(dataset))
for i in range(5):
    sample_X, sample_y = dataset[i]
    print(i, sample_X.shape, sample_y)

16242
0 torch.Size([1275]) tensor(-19.0138)
1 torch.Size([1275]) tensor(-10.1610)
2 torch.Size([1275]) tensor(-9.3766)
3 torch.Size([1275]) tensor(-13.7764)
4 torch.Size([1275]) tensor(-8.5371)


Now we can use the DataLoader function to use the dataset in PyTorch

In [5]:
N = len(dataset)

# input_len = dataset[0]['CMat'].shape[0]
input_len = dataset[0][0].shape[0]
batch_size = 32

train_data, test_data = torch.utils.data.random_split(dataset, (int(N*0.85), N - int(N*0.85)))

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data)

Now we can start to build our NN.

In [6]:
from torch import nn

class SimpleNN(nn.Module):
    
    def __init__(self):
        super(SimpleNN, self).__init__()
        # The input is a 1275 vector
        self.linear_relu_block = nn.Sequential(
            nn.Linear(input_len, 16, bias=False),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
            )
    
    def forward(self, x):
        logits = self.linear_relu_block(x)
        return logits
    
    
model = SimpleNN()
model.eval()

# Check if it works
random_result = model(torch.rand(input_len))
print(random_result.shape)
# print(model(next(iter(train_dataloader))[0]).shape)

torch.Size([1])


Now we can train the model. We use the SGD method, since I don't know how the rest work (TODO: Learn the others).

In [7]:
learning_rate = 1e-3
epochs = 2

SGD_opt = torch.optim.SGD(model.parameters(), lr=learning_rate)
MSE_fn = nn.MSELoss()

def train_epoch(train_data, model, optimizer, loss_fn):
    for batch, (X, y) in enumerate(train_data):
        # Prediction and loss
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        # Calculate gradient and backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            print(f'Current {batch * batch_size:6d} / {N:6d}. Train MSE = {loss:4.4f}', end='\r')
    return loss

def test_epoch(test_data, model, loss_fn):
    N = len(test_data.dataset)
    N_batch = N // len(test_data)
    Mse = 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(test_data):
            y_pred = model(X)
            Mse += loss_fn(y_pred, y).item()
    print(f'\nTest MSE {Mse / N_batch:4.4f}')
    return Mse / N_batch

def train_loop(train_data, test_data, model, optimizer, loss_fn, epochs=1):
    train_mse = []
    test_mse = []
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-'*30)
        
        tr_mse = train_epoch(train_data, model, optimizer, loss_fn)
        te_mse = test_epoch(test_data, model, loss_fn)
        
        train_mse.append(tr_mse)
        test_mse.append(te_mse)
    return train_mse, test_mse

We have defined the model and its training loop. Let's put it to use

In [8]:
train_mse, test_mse = train_loop(train_dataloader, test_dataloader, model, SGD_opt, MSE_fn, epochs=epochs)

Epoch 1/2
------------------------------


/home/eloi/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


/home/eloi/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/eloi/.local/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test MSE 82056.7295
Epoch 2/2
------------------------------
Test MSE 40428.98966242. Train MSE = 11.1886
